# Context Managers

Context managers are normally used to allocate and release resources automatically without needing to worry about about doing it yourself. You normally use the `with` keyword when dealing with CMs.

The most common example of a CM is when working with file IO. \
We show an example below:

In [1]:
with open('static/Genesis.txt', 'r') as f:
    print(next(f))
    print(next(f))

# will fail because the file has been closed
next(f)

[1:1] In the beginning when God created the heavens and the earth,

[1:2] the earth was a formless void and darkness covered the face of the deep, while a wind from God swept over the face of the waters.



ValueError: I/O operation on closed file.

Here we open the file, print the first two lines, and then close the file. The `open` CM handles opening and closing the file for you. The file is opened when you enter the `with` block and is closed when exiting the block. You can see that this is cleaner then manually opening and closing the file like below. Which is also prone to errors like forgetting to close the file yourself.

In [2]:
f = open('static/Genesis.txt', 'r')
print(next(f))
print(next(f), end='')
f.close()

[1:1] In the beginning when God created the heavens and the earth,

[1:2] the earth was a formless void and darkness covered the face of the deep, while a wind from God swept over the face of the waters.


***
In general, CMs are used to automatically run some sort of setup and then teardown code.

We can thus leverage our own CMs to automatically run some "setup" and "teardown" to do any number of tasks.

There are two main ways to create a Context Manager:
1. By defining a class.
1. By defining a generator.
***
When defining a class to build a CM there are two main methods we need to implement, the `__enter__` and the `__exit__` both define what happens when we *enter* and *exit* the `with` block.

Below we'll create a simple timer CM that just prints out how many seconds it takes to run the code in the block. When we enter the block we'll start the timer and when we exit the block we end the timer.

The `__exit__` method takes in three additional arguments for exception handling. If an exception occurs anywhere in the block the `__exit__` method is called and depending on your implementation either an exception is raised or the code handles it and continues. If at the end of `__exit__` we return `True` then if an exception occurs the code will handle it in the `__exit__` (if you choose to handle it) and continue after the with block.

In [57]:
from time import time

class Timer:
    def __enter__(self):
        self.start = time()
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.end = time()
        print(f'{self.end - self.start:.4f} seconds to run')
        
        if exc_type is not None:
            print('An exception happened:', exc_type, exc_value, exc_traceback)
        return True

In [60]:
with Timer():
    res = 0
    # sum up all values between 1 and 10,000,000
    for i in range(1, 10_000_001):
        if i == 5_000_000:
            raise Exception('AHH an exception!')
            print('moore')
        res += i
    
print()
print(f'Result Sum: {res:,}')
print(f'Expected Sum: {10_000_000*10_000_001 // 2:,}')

0.8478 seconds to run
An exception happened: <class 'Exception'> AHH an exception! <traceback object at 0x111f05bc0>

Result Sum: 12,499,997,500,000
Expected Sum: 50,000,005,000,000


The second way of defining a CM is by creating generator and decorating it with the `contextmanager` from `contextlib` which is part of the python standard library. Everyting before the `yield` is our `__enter__` and everything after is our `__exit__`.

In [63]:
from contextlib import contextmanager

@contextmanager
def timer():
    start = time()
    yield
    end = time()
    print(f'{end - start:.4f} seconds to run')

In [65]:
with timer():
    res = 0
    for i in range(10_000_000):
        res += i

1.4670 seconds to run


We'll also create a CM that mimics `open`, first by creating a class then with a generator.

In [66]:
class open_file:
    
    def __init__(self, filename, mode='r'):
        self.filename = filename
        self.mode = mode
        
    def __enter__(self):
        self.file = open(self.filename, self.mode)
        # notice we return the file we just opened
        # can you guess why / what this does?
        # if it's not clear now it should be in the next cell.
        return self.file
    
    # we can collapse the three arguments into 1 using *
    def __exit__(self, *excps):
        self.file.close()
        if excps[0] is not None:
            print(excps)
        return True

In [68]:
# when we return the file in the __enter__
# we can capture it in the with statement
with open_file('static/Genesis.txt') as f:
    print(next(f))
    print(next(f))
    
print(next(f))

[1:1] In the beginning when God created the heavens and the earth,

[1:2] the earth was a formless void and darkness covered the face of the deep, while a wind from God swept over the face of the waters.



ValueError: I/O operation on closed file.

Again using a generator.

In [70]:
@contextmanager
def open_file2(filename, mode='r'):
    file = open(filename, mode)
    # we yield file for the same reason as above
    yield file
    file.close()

In [71]:
with open_file2('static/Genesis.txt') as f:
    print(next(f))
    print(next(f))

[1:1] In the beginning when God created the heavens and the earth,

[1:2] the earth was a formless void and darkness covered the face of the deep, while a wind from God swept over the face of the waters.

